# Reverse Stress Testing

This notebook illustrates the method described in the paper: Y. Kopeliovich, A. Novosyolov, D. Satchkov, B. Schachter "Robust Risk Estimation and Hedging: A Reverse Stress Testing Approach".

Direct stress testing consists in setting a number of stress scenarios (unusual factor values, outliers), and computing portfolio loss under those scenarios.

Reverse stress testing problem formulates as follows. Given a portfolio and a predefined loss size, determine which factors stress (scenarios) would lead to that loss. This problem has infinitely many solutions, so additionally one would request that the scenarios selected would be <b>plausible</b> in some sense, and be sufficiently <b>different</b>.

## Method outline

Denote $X=(X_1,\dots,X_n)'$ a vector of factor returns, we assume that $X$ follows multivariate normal distribution with zero means $EX=0$ and covariance matrix $C=E(XX')$. Next, let $w=(w_1,\dots,w_n)'$ be a vector of factor weights in a portfolio, so that the portfolio return equals $R=w'X$. Clearly portfolio mean and variance are $ER=0$ and $VR=w'Cw$.

Let $L$ be the required loss size. Any scenario $x$ with $w'x=L$ provides this loss. To select most likely among these scenarios we need to maximize the conditional density function in $x$ given $w'x=L$, which is equivalent to minimizing $x'C^{-1}x$. Thus the most likely scenario $x=(x_1,\dots,x_n)'$ providing the required loss is the solution of the optimization problem

$$
x'C^{-1}x\to\min_x,
$$
$$
w'x=L.
$$

Solving it using Lagrange multipliers gives

$$
x_*=L\,\frac{Cw}{w'Cw}.
$$

Besides this most likely scenario, we would like to select a few more scenarios $x$ satisfying the equation $w'x=L$, located as far as possible from each other and from $x_*$, yet being quite possible. We formalize these conflicting requirements as follows. Select threshold $q$, and require that likelihood of all selected scenarios is not less than $q$ times the likelihood of $x_*$. In other words, we require that the <b>relative likelihood</b> varies from $q$ to 1.

## Code to be used

### Import tools

In [1]:
import pandas as pd
%pylab inline
from scipy.stats import multivariate_normal

Populating the interactive namespace from numpy and matplotlib


### Define auxiliary functions

In [2]:
def gram_schmidt_columns(x):
    q, r = np.linalg.qr(x)
    return q


def gram_schmidt_rows(x):
    """ row-wise Gram Schmidt procedure """
    q, r = np.linalg.qr(x.T)
    return q.T


def p_from_w(w):
    """ given weights vector w, calculate transform matrix p,
    as described in Appendix A of the paper
    """
    w = np.array(w)
    n = len(w)
    p = np.zeros((n, n))
    p[0, :] = w / np.linalg.norm(w)
    j0 = (np.abs(w) > 0.00001).nonzero()[0][0]
    for j in range(j0):
        p[j + 1, j] = 1
    for j in range(j0 + 1, n):
        p[j, j] = 1
    gs = gram_schmidt_rows(p)
    for j in range(gs.shape[1]):
        if gs[0, j] < 0:
            gs[:, j] = -gs[:, j]
    return gs


def standard_simplex_vertices(M):
    """ just that """
    z = np.eye(M) - 1 / M
    a = np.eye(M)
    a[0, :] = 1
    p = gram_schmidt_rows(a)
    cls = np.append(range(1, M), 0)
    gs = np.delete(p.dot(z), 0, axis=0)[:, cls]
    if gs[0, 0] < 0:
        gs = -gs
    return gs * np.sqrt(M / (M - 1))


def quadratic_form_inv(a, x):
    """ x * a^(-1) * x """
    ax = np.linalg.solve(a, x)
    return x.T.dot(ax)


### Main procedure

In [3]:
def comp_scenarios(c, w, L, q, M, printIt=False):
    """ given covariance matrix c, weights vector w,
    loss L, relative likelihood threshold q,
    and a number of non-central scenarios M,
    compute stress scenarios, and their relative likelihood;
    print all intermediate results if necessary
    """
    # central scenario
    cw = c.dot(w)
    ah = L * cw / w.dot(cw)
    if printIt:
        print('------------   ah   ------------')
        print(ah.round(3))

    # prepare transform matrix
    p = p_from_w(w)
    n = len(w)
    if printIt:
        print('------------   p   ------------')
        print(p.round(3))

    ahy_full = p.dot(ah)
    first_comp = ahy_full[0]
    if printIt:
        print('------------   ahy_full   ------------')
        print(ahy_full.round(3))

    # asset names
    ass = c.columns

    # compute covariance matrix in the transformed space
    dy = p.dot(c.dot(p.T))
    if printIt:
        print('------------   dy   ------------')
        print(dy.round(3))

    # extract decomposition parts
    d11 = dy[0, 0]
    # d1I = dy[0, 1:]
    dI1 = dy[1:, 0]
    dII = dy[1:, 1:]

    # compute center and conditional distribution in transformed space
    wn = np.linalg.norm(w)
    ahy = L * dI1 / d11 / wn
    i_m = np.matrix(dI1).reshape(n - 1, 1) * np.matrix(dI1).reshape(1, n - 1)
    dyh = dII - i_m / d11
    if printIt:
        print('------------   ahy   ------------')
        print(ahy.round(3))
        print('------------   dyh   ------------')
        print(dyh.round(3))

    # egenvalues in ascending order and eigenvectors
    vals, vect = np.linalg.eig(dyh)
    ind = vals.argsort()[::-1]
    V = vect[:, ind]

    # place the eigenvectors in opposite order
    # V = vect[:, ::-1]
    if printIt:
        print('------------   V   ------------')
        print(V.round(3))
        print('------------   Vals   ------------')
        print(vals.round(3))

    # compute vertices of the regular M-simplex
    t = standard_simplex_vertices(M)
    if printIt:
        print('------------   t   ------------')
        print(t.round(3))

    # rotation to principle components space
    z = V[:, :(M - 1)].dot(t)
    if printIt:
        print('------------   z   ------------')
        print(z.round(3))

    # compute the sphere radius
    z_last = z[:, -1]
    denom = quadratic_form_inv(dyh, np.array(z_last))
    r = np.sqrt(-2 * np.log(q) / denom)[0, 0]
    ahyr = ahy.repeat(M).reshape(n - 1, M)
    rz = r * z + ahyr
    if printIt:
        print('------------   r   ------------')
        print(r.round(3))

    # unite central scenario with sphere scenarios
    df = pd.DataFrame(ahy, columns=['aa']).join(pd.DataFrame(rz))
    cls = ['Scenario_{0}'.format(i) for i in range(M + 1)]
    df.columns = cls

    # append first_comp row at the beginning
    df0 = first_comp * pd.DataFrame(np.ones((1, M + 1)), columns=cls)
    df = df0.append(df.set_index(np.array(range(1, n))))
    if printIt:
        print('------------   df0   ------------')
        print(df0.round(3))
        print('------------   df   ------------')
        print(df.round(3))

    # rotating back to initial space
    ar = p.T.dot(df)

    # turning to dataframe
    scenarios = pd.DataFrame(ar, columns=cls).set_index(ass)
    if printIt:
        print('------------   scenarios  ------------')
        print(scenarios.round(3))

    lik = np.array([multivariate_normal.pdf(scenarios[cls[j]],
                                            mean=np.zeros((n,)),
                                            cov=c)
                    for j in range(M + 1)])
    lik = pd.DataFrame((lik / lik[0]).reshape(1, M + 1), columns=cls)
    lik.index = ['Likelihood']
    if printIt:
        print('------------   lik  ------------')
        print(lik.round(3))

    return scenarios, lik


### Example from Appendix 3 of the paper

In [4]:
ass = pd.Series(['Australia', 'Oil', 'Debt fund'])
ca = np.array([[11.871, 9.742, 1.125], [9.742, 14.723, 0.717], [1.125, 0.717, 0.219]])
c = pd.DataFrame(ca, columns=ass).set_index(ass)
w = np.ones((len(ass),))
M = 3
L = 13
q = 0.1
df, lik = comp_scenarios(c, w, L, q, M)

##### Factor covariance matrix

In [5]:
print(c)

           Australia     Oil  Debt fund
Australia     11.871   9.742      1.125
Oil            9.742  14.723      0.717
Debt fund      1.125   0.717      0.219


##### Stress scenarios

In [6]:
print(df.round(3))

           Scenario_0  Scenario_1  Scenario_2  Scenario_3
Australia       5.914       5.311       5.815       6.616
Oil             6.550       7.252       5.947       6.451
Debt fund       0.536       0.437       1.238      -0.067


##### Scenarios relative likelihood

In [7]:
print(lik.round(3))

            Scenario_0  Scenario_1  Scenario_2  Scenario_3
Likelihood         1.0       0.885         0.1         0.1


##### Check distances

In [8]:
np.array([np.linalg.norm(df.ix[:, i] - df.ix[:, j]) for i in range(M) for j in range(i+1, M+1)]).round(3)

array([ 0.931,  0.931,  0.931,  1.613,  1.613,  1.613])

Distances from the central scenario equal 0.931, distances between circle scenarios equal 1.613, as required.

### One more example

In [9]:
ass5 = pd.Series(['First', 'Second', 'Third', 'Fourth', 'Fifth'])
ca5 = np.array([[3.266, 0.767, 0.021, 0.977, 0.337],
                [0.767, 3.115, 0.023, 2.547, 0.351],
                [0.021, 0.023, 0.002, 0.022, 0.008],
                [0.977, 2.547, 0.022, 3.310, 0.339],
                [0.337, 0.351, 0.008, 0.339, 0.270]])
c = pd.DataFrame(ca5, columns=ass5).set_index(ass5)
w = np.ones((len(ass5),))
L = -5
M = 4
q = 0.01
scenarios, lik = comp_scenarios(c, w, L, q, M)

##### Factor covariance matrix

In [10]:
print(c)

        First  Second  Third  Fourth  Fifth
First   3.266   0.767  0.021   0.977  0.337
Second  0.767   3.115  0.023   2.547  0.351
Third   0.021   0.023  0.002   0.022  0.008
Fourth  0.977   2.547  0.022   3.310  0.339
Fifth   0.337   0.351  0.008   0.339  0.270


##### Stress scenatios

In [11]:
print(scenarios.round(3))

        Scenario_0  Scenario_1  Scenario_2  Scenario_3  Scenario_4
First       -1.294      -2.688      -0.823      -1.287      -0.377
Second      -1.640      -0.870      -0.809      -2.910      -1.968
Third       -0.018      -0.021      -0.012       0.002      -0.043
Fourth      -1.734      -1.056      -3.172      -1.664      -1.044
Fifth       -0.315      -0.365      -0.183       0.859      -1.568


##### Relative likelihood

In [12]:
print(lik.round(3))

            Scenario_0  Scenario_1  Scenario_2  Scenario_3  Scenario_4
Likelihood         1.0       0.592       0.126        0.01        0.01
